Homework 4: Sentiment Analysis - Task 4
----

Names
----
Names: __YOUR NAMES HERE__ (Write these in every notebook you submit)

Task 4: Neural Networks (20 points)
----

Next, we'll train a feedforward neural net to work with this data. You'll train one neural net which takes the same input as your Logistic Regression model - a sparse vector representing documents as bags of words.

Take a look at these videos to understand forward and backward propagation in neural networks - 
* https://www.youtube.com/watch?v=HHbjpDHcJVw
* https://youtu.be/-Lavz_I4l2U?si=zi20DB3qKPLMEPt1

You will be implementing **binarized** (presence/absence of word) and **multinomial** (counts of word) BoW representations of your data
  
**10 points in Task 5 will be allocated for all 9 graphs (including the one generated here in Task 4 for Neural Networks) being:**
- Legible
- Present below
- Properly labeled
     - x and y axes labeled
     - Legend for accuracy measures plotted
     - Plot Title with which model and run number the graph represents

In [15]:
!pip install --upgrade pip setuptools wheel
!pip install tensorflow-macos==2.13.1 tensorflow-metal
!pip install numpy pandas matplotlib jupyterlab nltk
!python -m pip uninstall -y sklearn
!python -m pip install -U scikit-learn

# !python -c "import tensorflow as tf; print(tf.__version__); print(tf.config.list_physical_devices())"

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 2.9 MB/s  0:00:03 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 2.8 MB/s  0:00:11m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [28]:
import sentiment_utils as sutils
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

# you can experiment with having some Dropout layers if you'd like to
# this is not required
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

# if you want to use this again
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer


In [33]:
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
DEV_FILE = "movie_reviews_dev.txt"

# load in your data and make sure you understand the format
# Do not print out too much so as to impede readability of your notebook
train_tups = sutils.generate_tuples_from_file(TRAIN_FILE)
dev_tups = sutils.generate_tuples_from_file(DEV_FILE)



# you may use either your sparse vectors or sklearn's CountVectorizer's sparse vectors
# you will experiment with multinomial and binarized representations later

In [34]:
# Create a feedforward neural network model
# that takes a sparse BoW representation of the data as input
# and makes a binary classification of positive/negative sentiment as output
# you may use any number of hidden layers >= 1 and any number of units in each hidden layer (we recommend between 50-200)
# you may use any activation function on the hidden layers 
# you should use a sigmoid activation function on the output layer
# you should use binary cross-entropy as your loss function
# sgd is an appropriate optimizer for this task
# you should report accuracy as your metric
# you may add Dropout layers if you'd like to

# create/compile your model in this cell

# Create a feedforward neural network model
model = Sequential()
model.add(Dense(128, input_dim=2000, activation='relu'))  # Adjust input_dim based on vocab size
model.add(Dropout(0.2))  # Dropout to avoid overfitting
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Display the model summary
model.summary()

# Compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train your model on the training data (using the pre-processed data from sentiment_utils)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 128)               256128    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 129       
                                                                 
Total params: 256257 (1001.00 KB)
Trainable params: 256257 (1001.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


How many trainable parameters does your model have? __YOUR ANSWER HERE__

In [35]:
# Convert the training data into features using featurization
train_X, train_y = sutils.featurize(train_tups, binary=True)  # For binarized representation
dev_X, dev_y = sutils.featurize(dev_tups, binary=True)

# Function to train the model and evaluate it, then return the accuracy
def train_and_evaluate_nn(train_X, train_y, dev_X, dev_y, epochs=8, batch_size=32):
    model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size)
    dev_loss, dev_accuracy = model.evaluate(dev_X, dev_y)
    return dev_accuracy

# Function to create and save the training graph for a specific run
def plot_neural_network_results(run_id, percents, save_as='NeuralNet_graph.png'):
    results = []
    
    for pct in percents:
        train_X_pct, train_y_pct = sutils.take_percent(train_X, train_y, pct)
        # Recreate the model each time to ensure it doesn't reuse trained weights
        model = Sequential()
        model.add(Dense(128, input_dim=2000, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Train and evaluate the model
        accuracy = train_and_evaluate_nn(train_X_pct, train_y_pct, dev_X, dev_y)
        results.append((pct, accuracy))

    # Plot the results
    x, y = zip(*results)
    plt.plot(x, y, label=f'Run {run_id} Accuracy')
    plt.xlabel('Training Data (%)')
    plt.ylabel('Accuracy')
    plt.title(f'Neural Network Accuracy vs. Training Data Size (Run {run_id})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(save_as, bbox_inches='tight', dpi=140)
    plt.close()

# Run the model three times, generating a plot each time
for run_id in range(1, 4):
    plot_neural_network_results(run_id, percents=[10, 20, 40, 60, 80, 100], save_as=f"NeuralNet_Run{run_id}.png")

TypeError: featurize() missing 1 required positional argument: 'data_X'

In [ ]:
# Use the model.evaluate function to report the loss and accuracy on the dev set
dev_loss, dev_accuracy = model.evaluate(dev_X, dev_y)
print(f"Loss on dev set: {dev_loss:.4f}")
print(f"Accuracy on dev set: {dev_accuracy:.4f}")


In [ ]:
# Create a graph similar to the one you made for Naive Bayes and Logistic Regression
# Track performance across different training sizes and save the graph

# Function to create and save training graphs
def plot_neural_network_results(metrics_fun, percents, save_as='NeuralNet_graph.png'):
    results = []
    
    for pct in percents:
        train_X_pct, train_y_pct = sutils.take_percent(train_X, train_y, pct)
        model.fit(train_X_pct, train_y_pct, epochs=8, batch_size=32)
        
        loss, accuracy = model.evaluate(dev_X, dev_y)
        results.append((pct, accuracy))

    # Plot the results
    x, y = zip(*results)
    plt.plot(x, y, label='Accuracy')
    plt.xlabel('Training Data (%)')
    plt.ylabel('Accuracy')
    plt.title('Neural Network Accuracy vs. Training Data Size')
    plt.legend()
    plt.savefig(save_as, bbox_inches='tight', dpi=140)
    plt.show()

# Call the function to plot the graph for the neural network
plot_neural_network_results(None, percents=[10, 20, 40, 60, 80, 100])


<span style="color: red;">__Expected Behavior__ </span>

**Neural Networks**:
Neural networks initialize their weights randomly and learn through iterative stochastic optimization, which introduces non-determinism by design. Even with the same data and parameters, different runs may lead to slightly different learned weights and therefore different graphs. In this case, variation between your three runs is expected and desired, it shows how the model’s training process can vary due to randomness.

<span style="color: red;">__Note on Training Data Increments__ </span>

When varying the amount of training data, choose increments that are meaningful and reasonable, you should be able to observe clear trends without making the experiment unnecessarily long. You may increment the training data percentage by **5%**, **10%** or **20%**.

**Make sure that one of your experiments includes 10% of the training data, as you will need this result to answer a question in Task 5.**

In [32]:
# create the same graph as with NB and LR, with your neural network model instead!
# make sure to re-create your model each time you train it — you don't want to start with
# an already trained network!

# you should experiment with different numbers of epochs to see how performance varies
# you need not create an experiment that takes > 10 min to run (gradescope will run out of computing resources and give you a 0)
# 生成与 NB/LR 相同的 4 指标曲线图
import matplotlib.pyplot as plt

def train_eval_nn_for_percent(percent: int,
                              use_count_vectorizer: bool,
                              binary_feats: bool,
                              epochs: int,
                              batch_size: int,
                              run_id: int = 1):
    """
    给定训练比例 -> 训练并在 dev 上返回 (P,R,F1,Acc)
    注意每次都重建模型，避免“续训”污染。
    """
    sub_X, sub_y = sutils.take_percent(train_X, train_y, pct=percent, shuffle=True, seed=run_id)
    X_tr, X_dv, vec = vectorize(use_count_vectorizer=use_count_vectorizer, binary=binary_feats, sub_X=sub_X)
    X_tr_arr = X_tr.astype('float32').toarray()
    X_dv_arr = X_dv.astype('float32').toarray()
    y_tr_arr = np.array(sub_y, dtype='float32')

    model = build_model(input_dim=X_tr_arr.shape[1], hidden_units=128, hidden_layers=1, dropout=0.2)
    model.fit(X_tr_arr, y_tr_arr, epochs=epochs, batch_size=batch_size, verbose=0)

    probs = model.predict(X_dv_arr, verbose=0).reshape(-1)
    preds = (probs >= 0.5).astype(int).tolist()
    return sutils.get_prfa(dev_y, preds, verbose=False)  # (P,R,F1,Acc)

# —— 设置实验组合（可改）
USE_COUNTVECT = True   # True=CountVectorizer；False=自定义
BINARY_FEATS  = False  # False=multinomial；True=binarized
EPOCHS        = 8
BATCH_SIZE    = 32
RUN_ID        = 1      # 记得做 1/2/3 三次

percents = [10, 20, 40, 60, 80, 100]  # 含 10%，满足 Task 5 要求
title = f"Neural Net ({'CountVectorizer' if USE_COUNTVECT else 'Custom'} | {'Binarized' if BINARY_FEATS else 'Multinomial'}) — Run {RUN_ID}"
savepath = f"NeuralNet_{'cv' if USE_COUNTVECT else 'custom'}_{'bin' if BINARY_FEATS else 'multi'}_run{RUN_ID}.png"

sutils.create_training_graph(
    metrics_fun=lambda p: train_eval_nn_for_percent(
        p, use_count_vectorizer=USE_COUNTVECT, binary_feats=BINARY_FEATS,
        epochs=EPOCHS, batch_size=BATCH_SIZE, run_id=RUN_ID
    ),
    percents=percents,
    title=title,
    savepath=savepath,
    verbose=False
)
print("Saved:", savepath)



2025-10-23 08:35:47.887640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:48.278213: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:48.495891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:48.997275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:49.357145: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:50.070642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-23 08:35:50.679114: I tensorflow/core/grappler/optimizers/cust

Saved: NeuralNet_cv_multi_run1.png


2025-10-23 08:35:55.473553: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Report the f1 scores for your model with the following settings, using the same number of epochs to train in both cases:
- number of epochs used: __YOUR ANSWER HERE__
- multinomial features: __YOUR ANSWER HERE__ 
- binarized features: __YOUR ANSWER HERE__

In [23]:
# # -*- coding: utf-8 -*-
# # Task 4: Feedforward Neural Network on BoW
# import numpy as np
# import sentiment_utils as sutils

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction import DictVectorizer

# # from keras.models import Sequential
# from tensorflow.keras.models import Sequential
# # from keras.layers import Dense, Dropout
# from tensorflow.keras.layers import Dense, Dropout
# # from keras.optimizers import SGD
# from tensorflow.keras.optimizers import SGD

# TRAIN_FILE = "movie_reviews_train.txt"
# DEV_FILE   = "movie_reviews_dev.txt"

# train_X, train_y = sutils.generate_tuples_from_file(TRAIN_FILE)
# dev_X,   dev_y   = sutils.generate_tuples_from_file(DEV_FILE)

# vocab = sutils.create_index(train_X, min_freq=1)

# def vectorize(use_cv: bool, binary: bool, sub_X=None):
#     if sub_X is None:
#         sub_X = train_X
#     if use_cv:
#         docs_train = [" ".join(t) for t in sub_X]
#         vec = CountVectorizer(binary=binary, vocabulary=None)
#         Xtr = vec.fit_transform(docs_train)
#         vec = CountVectorizer(binary=binary, vocabulary=vec.vocabulary_)
#         Xdv = vec.transform([" ".join(t) for t in dev_X])
#         return Xtr, Xdv, vec
#     else:
#         feats_tr = sutils.featurize(vocab, sub_X, binary=binary)
#         feats_dv = sutils.featurize(vocab, dev_X, binary=binary)
#         dv = DictVectorizer(sparse=True)
#         Xtr = dv.fit_transform(feats_tr)
#         Xdv = dv.transform(feats_dv)
#         return Xtr, Xdv, dv

# def build_model(input_dim: int, hidden_units: int = 128, hidden_layers: int = 1, dropout: float = 0.2):
#     model = Sequential()
#     # First hidden
#     model.add(Dense(hidden_units, activation='relu', input_shape=(input_dim,)))
#     if dropout > 0:
#         model.add(Dropout(dropout))
#     for _ in range(hidden_layers - 1):
#         model.add(Dense(hidden_units, activation='relu'))
#         if dropout > 0:
#             model.add(Dropout(dropout))
#     # Output
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.05, momentum=0.9), metrics=['accuracy'])
#     return model

# def train_eval_nn(percent: int, use_cv: bool, binary: bool, epochs: int, batch_size: int, run_id: int = 1):
#     sub_X, sub_y = sutils.take_percent(train_X, train_y, percent, shuffle=True, seed=run_id)
#     Xtr, Xdv, vec = vectorize(use_cv=use_cv, binary=binary, sub_X=sub_X)

#     input_dim = Xtr.shape[1]
#     model = build_model(input_dim=input_dim, hidden_units=128, hidden_layers=1, dropout=0.2)
#     # Keras expects dense arrays; convert sparse to dense if tiny, else to csr then toarray() carefully
#     Xtr_arr = Xtr.astype('float32').toarray()
#     Xdv_arr = Xdv.astype('float32').toarray()
#     ytr = np.array(sub_y, dtype='float32')

#     model.fit(Xtr_arr, ytr, epochs=epochs, batch_size=batch_size, verbose=0)
#     # Evaluate & predict
#     dev_probs = model.predict(Xdv_arr, verbose=0).reshape(-1)
#     preds = (dev_probs >= 0.5).astype(int).tolist()
#     prec, rec, f1, acc = sutils.get_prfa(dev_y, preds, verbose=False)
#     return prec, rec, f1, acc, model.count_params()

# def plot_runs_nn(use_cv: bool, binary: bool, epochs: int, batch_size: int, run_id: int, percents=None):
#     if percents is None:
#         percents = [10, 20, 40, 60, 80, 100]
#     title = f"Neural Net ({'CountVectorizer' if use_cv else 'Custom'} | {'Binarized' if binary else 'Multinomial'}) — Run {run_id}"
#     sutils.create_training_graph(
#         metrics_fun=lambda p: train_eval_nn(p, use_cv=use_cv, binary=binary, epochs=epochs, batch_size=batch_size, run_id=run_id)[:4],
#         percents=percents,
#         title=title,
#         savepath=f"NeuralNet_{'cv' if use_cv else 'custom'}_{'bin' if binary else 'multi'}_run{run_id}.png"
#     )

# if __name__ == "__main__":
#     # Example default: multinomial with CountVectorizer
#     epochs = 8
#     batch = 32
#     # Report parameter count once @100%
#     _,_,f1,_, params = train_eval_nn(100, use_cv=True, binary=False, epochs=epochs, batch_size=batch, run_id=1)
#     print(f"Model trainable parameters: {params}")
#     print(f"F1 (epochs={epochs}) multinomial: {f1:.4f}")

#     for run in (1,2,3):
#         plot_runs_nn(use_cv=True, binary=False, epochs=epochs, batch_size=batch, run_id=run)


In [1]:
import sys
print(sys.version)

3.9.24 (main, Oct 21 2025, 20:06:27) 
[Clang 20.1.8 ]


In [2]:
import platform
print(platform.python_version())

3.9.24


In [3]:
import sys
print(sys.executable)

/opt/anaconda3/envs/MJQHW/bin/python


In [4]:
import os
print(os.environ.get('CONDA_DEFAULT_ENV'))

MJQHW


In [5]:
import sys
import os
import platform

print(f"🐍 Python 版本: {platform.python_version()}")
print(f"📁 解释器路径: {sys.executable}")
print(f"🌱 虚拟环境名称: {os.path.basename(os.path.dirname(sys.executable))}")
print(f"🔹 Conda 环境变量: {os.environ.get('CONDA_DEFAULT_ENV')}")

🐍 Python 版本: 3.9.24
📁 解释器路径: /opt/anaconda3/envs/MJQHW/bin/python
🌱 虚拟环境名称: bin
🔹 Conda 环境变量: MJQHW


In [6]:
import sys
import os
import platform

python_exe = sys.executable
env_name = os.path.basename(os.path.dirname(os.path.dirname(python_exe)))

print(f"🐍 Python 版本: {platform.python_version()}")
print(f"📁 解释器路径: {python_exe}")
print(f"🌱 虚拟环境名称: {env_name}")
print(f"🔹 Conda 环境变量: {os.environ.get('CONDA_DEFAULT_ENV')}")

🐍 Python 版本: 3.9.24
📁 解释器路径: /opt/anaconda3/envs/MJQHW/bin/python
🌱 虚拟环境名称: MJQHW
🔹 Conda 环境变量: MJQHW


In [7]:
!pip list

Package                   Version
------------------------- -----------
anyio                     4.10.0
appnope                   0.1.4
argon2-cffi               21.3.0
argon2-cffi-bindings      25.1.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     24.3.0
babel                     2.16.0
backcall                  0.2.0
beautifulsoup4            4.13.5
bleach                    6.2.0
brotlicffi                1.0.9.2
certifi                   2025.10.5
cffi                      2.0.0
charset-normalizer        3.3.2
comm                      0.2.1
debugpy                   1.8.16
decorator                 5.2.1
defusedxml                0.7.1
exceptiongroup            1.2.0
executing                 2.2.1
fastjsonschema            2.20.0
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
idna                      3.7
importlib_metadata        8.5.0
ipykernel                 6.30.1
ipython          